# [Housing Prices Competition for Kaggle Learn Users](https://www.kaggle.com/c/home-data-for-ml-course)

In [1]:
# Code you have previously used to load data
import pandas as pd
import numpy as np

from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

from sklearn import linear_model

from sklearn.metrics import mean_absolute_error, r2_score

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

## Загрузка данных

In [10]:
# данные, которые раньше были с nan
train_data_with_nan = pd.read_csv("./hpc_train_data_with_nan.csv") 
train_data_with_nan

,Unnamed: 0,SalePrice,LotFrontage,MasVnrArea,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Electrical,FireplaceQu,GarageYrBlt,GarageFinish,GarageCond,PoolQC,Fence
0,0,208500,65.0,196.0,4,3,1,6,1,4,0,2003.0,2,3,0,0
1,1,181500,80.0,0.0,4,3,4,5,1,4,3,1976.0,2,3,0,0
2,2,223500,68.0,162.0,4,3,2,6,1,4,3,2001.0,2,3,0,0
3,3,140000,60.0,0.0,3,4,1,5,1,4,4,1998.0,1,3,0,0
4,4,250000,84.0,350.0,4,3,3,6,1,4,3,2000.0,2,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1455,175000,62.0,0.0,4,3,1,1,1,4,3,1999.0,2,3,0,0
1456,1456,210000,85.0,119.0,4,3,1,5,3,4,3,1978.0,1,3,0,3
1457,1457,266500,66.0,0.0,3,4,1,6,1,4,4,1941.0,2,3,0,4
1458,1458,142125,68.0,0.0,3,3,2,6,3,3,0,1950.0,1,3,0,0


In [8]:
y = train_data_with_nan.SalePrice
y

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [9]:
train_data_with_nan = train_data_with_nan.drop(columns=['SalePrice'])
train_data_with_nan

,Unnamed: 0,LotFrontage,MasVnrArea,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Electrical,FireplaceQu,GarageYrBlt,GarageFinish,GarageCond,PoolQC,Fence
0,0,65.0,196.0,4,3,1,6,1,4,0,2003.0,2,3,0,0
1,1,80.0,0.0,4,3,4,5,1,4,3,1976.0,2,3,0,0
2,2,68.0,162.0,4,3,2,6,1,4,3,2001.0,2,3,0,0
3,3,60.0,0.0,3,4,1,5,1,4,4,1998.0,1,3,0,0
4,4,84.0,350.0,4,3,3,6,1,4,3,2000.0,2,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1455,62.0,0.0,4,3,1,1,1,4,3,1999.0,2,3,0,0
1456,1456,85.0,119.0,4,3,1,5,3,4,3,1978.0,1,3,0,3
1457,1457,66.0,0.0,3,4,1,6,1,4,4,1941.0,2,3,0,4
1458,1458,68.0,0.0,3,3,2,6,3,3,0,1950.0,1,3,0,0


In [4]:
# числовые данные, которые раньше были без nan
train_data_without_nan = pd.read_csv("./hpc_train_data_without_nan.csv") 
train_data_without_nan

,Unnamed: 0,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,0,60,8450,7,5,2003,2003,706,0,150,...,0,61,0,0,0,0,0,2,2008,208500
1,1,20,9600,6,8,1976,1976,978,0,284,...,298,0,0,0,0,0,0,5,2007,181500
2,2,60,11250,7,5,2001,2002,486,0,434,...,0,42,0,0,0,0,0,9,2008,223500
3,3,70,9550,7,5,1915,1970,216,0,540,...,0,35,272,0,0,0,0,2,2006,140000
4,4,60,14260,8,5,2000,2000,655,0,490,...,192,84,0,0,0,0,0,12,2008,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1455,60,7917,6,5,1999,2000,0,0,953,...,0,40,0,0,0,0,0,8,2007,175000
1456,1456,20,13175,6,6,1978,1988,790,163,589,...,349,0,0,0,0,0,0,2,2010,210000
1457,1457,70,9042,7,9,1941,2006,275,0,877,...,0,60,0,0,0,0,2500,5,2010,266500
1458,1458,20,9717,5,6,1950,1996,49,1029,0,...,366,0,112,0,0,0,0,4,2010,142125


In [ ]:
train_data = pd.DataFrame()
train_data = train_data.join(train_data_with_nan)
train_data = train_data.join(train_data_without_nan)